In [ ]:
!pip install atproto

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.2/416.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.1/301.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 0.8.3 requires websockets>=14.0, but you have websockets 13.1 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **fetch missing informations**

In [ ]:
from atproto import Client
from getpass import getpass
import os
import json
import time
import sys
from typing import List, Dict, Optional
from getpass import getpass

In [ ]:
DRIVE_FOLDER = "/content/drive/MyDrive/Nour/users_posts_json"
PAGE_LIMIT   = 100
SLEEP_SEC    = 0.2

In [ ]:
# 1) Collect handles from filenames inside the folder (strip .json)
def list_handles_from_folder(folder: str):
    handles = []
    for fn in os.listdir(folder):
        if not fn.lower().endswith(".json"):
            continue
        # Strip extension and normalize
        handle = os.path.splitext(fn)[0]
        handle = handle.replace("_", ".").lower()
        handles.append(handle)
    return sorted(set(handles))

# 2) ATProto helpers
def fetch_all_follow_handles(client: Client, actor: str, limit: int = PAGE_LIMIT, sleep_sec: float = SLEEP_SEC) -> List[str]:
    """Friends = accounts the actor follows."""
    out: List[str] = []
    cursor: Optional[str] = None
    while True:
        resp = client.app.bsky.graph.get_follows({'actor': actor, 'limit': limit, 'cursor': cursor})
        for p in resp.follows or []:
            h = getattr(p, 'handle', None)
            if h:
                out.append(h)
        cursor = getattr(resp, 'cursor', None)
        if not cursor:
            break
        time.sleep(sleep_sec)
    return out

def fetch_all_follower_handles(client: Client, actor: str, limit: int = PAGE_LIMIT, sleep_sec: float = SLEEP_SEC) -> List[str]:
    out: List[str] = []
    cursor: Optional[str] = None
    while True:
        resp = client.app.bsky.graph.get_followers({'actor': actor, 'limit': limit, 'cursor': cursor})
        for p in resp.followers or []:
            h = getattr(p, 'handle', None)
            if h:
                out.append(h)
        cursor = getattr(resp, 'cursor', None)
        if not cursor:
            break
        time.sleep(sleep_sec)
    return out

In [ ]:
# 3) Login
client = Client()
username_bsky = input("Enter your Bluesky username (handle or email): ")
password_bsky = getpass("Enter your Bluesky password: ")
client.login(username_bsky, password_bsky)

Enter your Bluesky username (handle or email): noourr.bsky.social
Enter your Bluesky password: ··········


ProfileViewDetailed(did='did:plc:lotsvgolb7viwjujkvtrvxhs', handle='noourr.bsky.social', associated=ProfileAssociated(activity_subscription=ProfileAssociatedActivitySubscription(allow_subscriptions='followers', py_type='app.bsky.actor.defs#profileAssociatedActivitySubscription'), chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:lotsvgolb7viwjujkvtrvxhs/bafkreib3uaqjhj2fxgjdaonsaphtxe2nc7fjfl6p3lw2wamknne7pmkax4@jpeg', banner=None, created_at='2025-06-24T16:23:36.632Z', description=None, display_name='', followers_count=6, follows_count=1, indexed_at='2025-06-24T16:23:36.632Z', joined_via_starter_pack=None, labels=[], pinned_post=None, posts_count=0, status=None, verification=None, viewer=ViewerState(activity_subscription=None, blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None,

In [ ]:
# 4) Get handles from Drive
handles = list_handles_from_folder(DRIVE_FOLDER)
if not handles:
    print(f"No .json files found in {DRIVE_FOLDER}. Make sure the folder and filenames look like handle.json")
    sys.exit(0)

print(f"Found {len(handles)} handles")

Found 5785 handles


In [ ]:
handles

['00dl3-aibfan.bsky.social',
 '022770.bsky.social',
 '0331online.bsky.social',
 '0jeff0.bsky.social',
 '10bmnews.bsky.social',
 '12sknnews.bsky.social',
 '13thmonthstudio.bsky.social',
 '14katiek.bsky.social',
 '15sksb7.bsky.social',
 '1692witchywoman.bsky.social',
 '1707unsubscribe.bsky.social',
 '1863photog.bsky.social',
 '18thcenturyboy.bsky.social',
 '1and1and1is3.bsky.social',
 '1bird3.bsky.social',
 '1fakt.bsky.social',
 '1naturistlife.bsky.social',
 '1stwark.bsky.social',
 '202fsunole.bsky.social',
 '2leavespothos.bsky.social',
 '2simplyput.bsky.social',
 '38boog.bsky.social',
 '3nomore.bsky.social',
 '3touraz.bsky.social',
 '3zmac.bsky.social',
 '4040rec.bsky.social',
 '404evr.com',
 '415goldblooded.bsky.social',
 '416raccoon.bsky.social',
 '4nier.bsky.social',
 '4rs0n.bsky.social',
 '50501arkansas.bsky.social',
 '515angel.bsky.social',
 '51skipper.bsky.social',
 '5newmanl.bsky.social',
 '5thgenfljane.bsky.social',
 '66talk.bsky.social',
 '670rv.bsky.social',
 '6canrebf.bsky.so

In [ ]:
import os
import pandas as pd

def collect_processed_handles(output_dir: str) -> set:
    """
    Scan all CSVs in output_dir, collect user_handle values (lowercased).
    Ignores files without 'user_handle' column or unreadable CSVs.
    """
    processed = set()
    if not os.path.isdir(output_dir):
        return processed

    for fname in os.listdir(output_dir):
        if not fname.lower().endswith(".csv"):
            continue
        fpath = os.path.join(output_dir, fname)
        try:

            df = pd.read_csv(fpath, usecols=["user_handle"])
            processed.update(df["user_handle"].astype(str).str.lower())
        except Exception as e:
            print(f"Skip {fpath} (no user_handle col or unreadable): {e}")
    return processed

def compute_remaining_handles(all_handles: list, output_dir: str) -> list:
    """
    Given all_handles (already normalized) and an output_dir of CSV batches,
    return the list of handles that are not yet processed.
    """
    done = collect_processed_handles(output_dir)
    remaining = [h for h in all_handles if h.lower() not in done]
    print(f"Total: {len(all_handles)} | Processed: {len(done)} | Remaining: {len(remaining)}")
    return remaining


OUTPUT_DIR = "/content/drive/MyDrive/Nour/bluesky/user_level"
remaining_handles = compute_remaining_handles(handles, OUTPUT_DIR)


Total: 5785 | Processed: 3774 | Remaining: 2011


In [ ]:
# 5) Process each user
rows = []
for idx, actor in enumerate(remaining_handles, 1):
    try:
        profile = client.app.bsky.actor.get_profile({'actor': actor})
        # core profile stats
        user_description     = getattr(profile, 'description', None)
        user_followers_count = getattr(profile, 'followers_count', None)
        user_friends_count   = getattr(profile, 'follows_count', None)
        posts_count          = getattr(profile, 'posts_count', None)
        user_did             = getattr(profile, 'did', None)
        canonical_handle     = getattr(profile, 'handle', actor)  # in case of handle redirect

        # lists (paginated)
        friend_handles   = fetch_all_follow_handles(client, canonical_handle)
        follower_handles = fetch_all_follower_handles(client, canonical_handle)

        # Pack lists as JSON strings so CSV stays clean
        row = {
            "user_handle": canonical_handle,
            "user_did": user_did,
            "user_description": user_description,
            "user_followers_count": user_followers_count,
            "user_friends_count": user_friends_count,
            "posts_count": posts_count,
            "user_follower_handles": json.dumps(follower_handles, ensure_ascii=False),
            "user_friend_handles": json.dumps(friend_handles, ensure_ascii=False),
        }
        rows.append(row)

        print(f"[{idx}/{len(remaining_handles)}] {canonical_handle}: "
              f"followers={user_followers_count}, friends={user_friends_count}, posts={posts_count}, "
              f"follower_handles={len(follower_handles)}, friend_handles={len(friend_handles)}")

    except Exception as e:
        print(f"[{idx}/{len(remaining_handles)}] {actor}: ERROR -> {e}")
        continue

import pandas as pd
df = pd.DataFrame(rows, columns=[
    "user_handle",
    "user_did",
    "user_description",
    "user_followers_count",
    "user_friends_count",
    "posts_count",
    "user_listed_count",
    "user_follower_handles",
    "user_friend_handles",
])
OUTPUT_CSV   = "/content/drive/MyDrive/Nour/bluesky/user_level/user_level_infos_10.csv"
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")

print(f"\nWrote {len(df)} rows to {OUTPUT_CSV}")


# **count labeled posts**

In [ ]:
import json
from collections import Counter

file_path = "/content/drive/MyDrive/Nour/bluesky/exports/filtered_posts_full.jsonl"

total_posts = 0
labeled_posts = 0
label_counter = Counter()

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        total_posts += 1
        data = json.loads(line)
        labels = data.get("post", {}).get("labels", [])
        if labels:
            labeled_posts += 1
            for lbl in labels:
                if isinstance(lbl, dict):
                    label_counter[lbl.get("val", str(lbl))] += 1
                else:
                    label_counter[str(lbl)] += 1

labeled_percentage = (labeled_posts / total_posts) * 100 if total_posts else 0
label_percentages = {label: (count / labeled_posts) * 100 for label, count in label_counter.items()}

print(f"Total posts: {total_posts}")
print(f"Labeled posts: {labeled_posts} ({labeled_percentage:.2f}%)\n")

print("Label distribution among labeled posts:")
for label, pct in label_percentages.items():
    print(f" - {label}: {pct:.2f}%")


Total posts: 3453
Labeled posts: 4 (0.12%)

Label distribution among labeled posts:
 - threat: 25.00%
 - graphic-media: 25.00%
 - Cancer-Vaccine: 25.00%
 - sexual: 25.00%


In [ ]:
import json
from collections import Counter
import pandas as pd

file_path = "/content/drive/MyDrive/Nour/bluesky/exports/filtered_posts_full.jsonl"

def extract_labels(item):
    """
    Extract labels from likely locations, prioritizing author-level labels as user indicated.
    Returns a set of normalized label strings for this record.
    """
    labels = set()
    post = item.get("post", {})

    # 0) Author-level labels (per user hint)
    author = post.get("author", {})
    a_labels = author.get("labels", [])
    if isinstance(a_labels, dict):
        # structure like {"values": [{"val": "..."}]}
        for v in a_labels.get("values", []):
            if isinstance(v, dict):
                val = v.get("val")
                if val:
                    labels.add(str(val).strip())
            elif isinstance(v, str):
                labels.add(v.strip())
    elif isinstance(a_labels, list):
        for v in a_labels:
            if isinstance(v, dict):
                if "val" in v: labels.add(str(v["val"]).strip())
                elif "label" in v: labels.add(str(v["label"]).strip())
            elif isinstance(v, str):
                labels.add(v.strip())

    # 1) Post-level moderation labels (fallback)
    p_labels = post.get("labels", [])
    if isinstance(p_labels, dict):
        for v in p_labels.get("values", []):
            if isinstance(v, dict) and "val" in v:
                labels.add(str(v["val"]).strip())
            elif isinstance(v, str):
                labels.add(v.strip())
    elif isinstance(p_labels, list):
        for v in p_labels:
            if isinstance(v, dict):
                if "val" in v: labels.add(str(v["val"]).strip())
                elif "label" in v: labels.add(str(v["label"]).strip())
            elif isinstance(v, str):
                labels.add(v.strip())

    # 2) Record self-labels (extra fallback)
    rec = post.get("record", {})
    rec_labels = rec.get("labels") or rec.get("selfLabels")
    if isinstance(rec_labels, dict):
        for v in rec_labels.get("values", []):
            if isinstance(v, dict) and "val" in v:
                labels.add(str(v["val"]).strip())
            elif isinstance(v, str):
                labels.add(v.strip())
    elif isinstance(rec_labels, list):
        for v in rec_labels:
            if isinstance(v, dict) and "val" in v:
                labels.add(str(v["val"]).strip())
            elif isinstance(v, str):
                labels.add(v.strip())

    # normalize & prune empties
    labels = {x for x in (l.lower() for l in labels) if x}
    return labels

total_posts = 0
labeled_posts = 0
label_post_counts = Counter()  # how many labeled posts contain each label

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        total_posts += 1
        try:
            item = json.loads(line)
        except json.JSONDecodeError:
            continue

        labels = extract_labels(item)
        if labels:
            labeled_posts += 1
            for lbl in labels:
                label_post_counts[lbl] += 1

labeled_pct = (labeled_posts / total_posts * 100.0) if total_posts else 0.0
print(f"Total posts: {total_posts}")
print(f"Labeled posts : {labeled_posts} ({labeled_pct:.2f}%)")

# Build a table for the user
if labeled_posts > 0:
    rows = []
    for lbl, count in sorted(label_post_counts.items(), key=lambda x: (-x[1], x[0])):
        pct = (count / labeled_posts) * 100.0
        rows.append({"label": lbl, "posts_with_label": count, "pct_of_labeled_posts": round(pct, 2)})
    df = pd.DataFrame(rows)
    df.head()


Total posts: 3453
Labeled posts (any source, prioritizing author.labels): 280 (8.11%)


In [ ]:
df

,label,posts_with_label,pct_of_labeled_posts
0,!no-unauthenticated,202,72.14
1,rude,45,16.07
2,misinformation,43,15.36
3,bridged-from-bridgy-fed-activitypub,28,10.00
4,intolerant,3,1.07
5,cancer-vaccine,1,0.36
6,graphic-media,1,0.36
7,porn,1,0.36
8,sexual,1,0.36
9,threat,1,0.36


In [ ]:
import json
import csv
from collections import Counter
from pathlib import Path

# Hardcoded path for Colab
jsonl_path = Path("/content/drive/MyDrive/Nour/bluesky/exports/filtered_posts_full.jsonl")

# -----------------------------
# include the extract_all_labels_from_post function from earlier
# -----------------------------
def _normalize_label(x):
    if x is None:
        return None
    if isinstance(x, str):
        s = x.strip()
        return s.lower() if s else None
    return None

def _extract_from_labels_value(node):
    if node is None:
        return []
    results = []
    if isinstance(node, dict):
        values = node.get("values")
        if isinstance(values, list):
            for v in values:
                if isinstance(v, dict) and "val" in v:
                    lab = _normalize_label(v.get("val"))
                    if lab:
                        results.append(lab)
                elif isinstance(v, str):
                    lab = _normalize_label(v)
                    if lab:
                        results.append(lab)
        if "val" in node:
            lab = _normalize_label(node.get("val"))
            if lab:
                results.append(lab)
        if "label" in node:
            lab = _normalize_label(node.get("label"))
            if lab:
                results.append(lab)
    elif isinstance(node, list):
        for v in node:
            if isinstance(v, dict):
                if "val" in v:
                    lab = _normalize_label(v.get("val"))
                    if lab:
                        results.append(lab)
                if "label" in v:
                    lab = _normalize_label(v.get("label"))
                    if lab:
                        results.append(lab)
                if "value" in v and isinstance(v["value"], dict) and "val" in v["value"]:
                    lab = _normalize_label(v["value"]["val"])
                    if lab:
                        results.append(lab)
            elif isinstance(v, str):
                lab = _normalize_label(v)
                if lab:
                    results.append(lab)
    elif isinstance(node, str):
        lab = _normalize_label(node)
        if lab:
            results.append(lab)
    return results

def _recursive_label_scan(obj, labels):
    if isinstance(obj, dict):
        for k, v in obj.items():
            lk = k.lower()
            if "label" in lk:
                for lab in _extract_from_labels_value(v):
                    labels.add(lab)
                if isinstance(v, str):
                    lab = _normalize_label(v)
                    if lab:
                        labels.add(lab)
            if lk in {"labels", "selflabels"}:
                for lab in _extract_from_labels_value(v):
                    labels.add(lab)
            _recursive_label_scan(v, labels)
    elif isinstance(obj, list):
        for x in obj:
            _recursive_label_scan(x, labels)

def extract_all_labels_from_post(post_obj):
    labels = set()
    author = post_obj.get("author", {})
    if isinstance(author, dict):
        for lab in _extract_from_labels_value(author.get("labels")):
            labels.add(lab)
    for lab in _extract_from_labels_value(post_obj.get("labels")):
        labels.add(lab)
    record = post_obj.get("record", {})
    if isinstance(record, dict):
        for key in ("labels", "selfLabels"):
            for lab in _extract_from_labels_value(record.get(key)):
                labels.add(lab)
    _recursive_label_scan(post_obj, labels)
    labels = {l for l in (lab.strip().lower() for lab in labels) if l}
    return labels

# -----------------------------
# Main analysis
# -----------------------------
if not jsonl_path.exists():
    raise FileNotFoundError(f"File not found: {jsonl_path}")

total_posts = 0
labeled_posts = 0
label_post_counts = Counter()

with jsonl_path.open("r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            item = json.loads(line)
        except json.JSONDecodeError:
            continue

        total_posts += 1
        post = item.get("post", {})
        if not isinstance(post, dict):
            continue

        labels = extract_all_labels_from_post(post)
        if labels:
            labeled_posts += 1
            for lab in labels:
                label_post_counts[lab] += 1

labeled_pct = (labeled_posts / total_posts * 100.0) if total_posts else 0.0

print(f"Total posts: {total_posts}")
print(f"Labeled posts: {labeled_posts} ({labeled_pct:.2f}%)\n")

if labeled_posts > 0:
    rows = []
    for lab, count in sorted(label_post_counts.items(), key=lambda x: (-x[1], x[0])):
        pct_labeled = (count / labeled_posts) * 100.0
        pct_all = (count / total_posts) * 100.0 if total_posts else 0.0
        rows.append({
            "label": lab,
            "posts_with_label": count,
            "pct_of_labeled_posts": round(pct_labeled, 4),
            "pct_of_all_posts": round(pct_all, 4),
        })

    print("Label distribution:")
    for r in rows:
        print(f" - {r['label']}: {r['pct_of_labeled_posts']:.2f}% "
              f"({r['posts_with_label']}/{labeled_posts}) "
              f"[{r['pct_of_all_posts']:.2f}% of all posts]")


Total posts: 3453
Labeled posts: 330 (9.56%)

Label distribution:
 - !no-unauthenticated: 76.06% (251/330) [7.27% of all posts]
 - rude: 13.64% (45/330) [1.30% of all posts]
 - misinformation: 13.03% (43/330) [1.25% of all posts]
 - bridged-from-bridgy-fed-activitypub: 8.48% (28/330) [0.81% of all posts]
 - intolerant: 0.91% (3/330) [0.09% of all posts]
 - cancer-vaccine: 0.30% (1/330) [0.03% of all posts]
 - doi.org/g9j95s: 0.30% (1/330) [0.03% of all posts]
 - graphic-media: 0.30% (1/330) [0.03% of all posts]
 - porn: 0.30% (1/330) [0.03% of all posts]
 - sexual: 0.30% (1/330) [0.03% of all posts]
 - threat: 0.30% (1/330) [0.03% of all posts]


# **prepare the final csv of the relevant users**

In [ ]:
import json
import os
from pathlib import Path
import pandas as pd

# ---------- Paths (edit if needed) ----------
JSONL_PATH = Path("/content/drive/MyDrive/Nour/bluesky/exports/filtered_posts_full.jsonl")
CSV_DIR    = Path("/content/drive/MyDrive/Nour/bluesky/user_level/")
CSV_GLOB   = "user_level_infos_*.csv"   # matches 1..12 (or more)
OUTPUT_CSV = CSV_DIR / "filtered_users_from_jsonl.csv"

# ---------- 1) Collect author DIDs from JSONL ----------
author_dids = set()
if not JSONL_PATH.exists():
    raise FileNotFoundError(f"JSONL not found: {JSONL_PATH}")

with JSONL_PATH.open("r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            item = json.loads(line)
        except json.JSONDecodeError:
            continue
        # robustly fetch post.author.did
        post = item.get("post", {})
        author = post.get("author", {}) if isinstance(post, dict) else {}
        did = author.get("did")
        if isinstance(did, str) and did:
            author_dids.add(did)

print(f"Unique author DIDs found in JSONL: {len(author_dids):,}")

# ---------- 2) Scan CSVs and filter rows where user_did is in author_dids ----------
csv_files = sorted(CSV_DIR.glob(CSV_GLOB), key=lambda p: (p.stem, p.name))
if not csv_files:
    raise FileNotFoundError(f"No CSV files matching {CSV_GLOB} in {CSV_DIR}")

# Prepare output
if OUTPUT_CSV.exists():
    OUTPUT_CSV.unlink()  # start fresh

total_in_rows = 0
total_kept_rows = 0
header_written = False

for i, csv_path in enumerate(csv_files, 1):
    print(f"[{i}/{len(csv_files)}] Processing: {csv_path.name}")

    # Read in chunks for memory safety
    for chunk in pd.read_csv(csv_path, dtype=str, chunksize=100_000):
        total_in_rows += len(chunk)
        if "user_did" not in chunk.columns:
            raise KeyError(f"'user_did' column not found in {csv_path}")

        kept = chunk[chunk["user_did"].isin(author_dids)]
        total_kept_rows += len(kept)

        # Append to output
        kept.to_csv(
            OUTPUT_CSV,
            mode="a",
            index=False,
            header=not header_written
        )
        header_written = True

print("\n--- Summary ---")
print(f"Input CSV files: {len(csv_files)}")
print(f"Total rows scanned: {total_in_rows:,}")
print(f"Rows kept (user_did in JSONL authors): {total_kept_rows:,}")
print(f"Output written to: {OUTPUT_CSV}")


Unique author DIDs found in JSONL: 1,661
[1/12] Processing: user_level_infos_1.csv
[2/12] Processing: user_level_infos_10.csv
[3/12] Processing: user_level_infos_11.csv
[4/12] Processing: user_level_infos_12.csv
[5/12] Processing: user_level_infos_2.csv
[6/12] Processing: user_level_infos_3.csv
[7/12] Processing: user_level_infos_4.csv
[8/12] Processing: user_level_infos_5.csv
[9/12] Processing: user_level_infos_6.csv
[10/12] Processing: user_level_infos_7.csv
[11/12] Processing: user_level_infos_8.csv
[12/12] Processing: user_level_infos_9.csv

--- Summary ---
Input CSV files: 12
Total rows scanned: 6,620
Rows kept (user_did in JSONL authors): 1,194
Output written to: /content/drive/MyDrive/Nour/bluesky/user_level/filtered_users_from_jsonl.csv


In [ ]:
JSONL_PATH = "/content/drive/MyDrive/Nour/bluesky/user_level/filtered_users_from_jsonl.csv"
df_users=pd.read_csv(JSONL_PATH)
df_users.head()


,user_handle,user_did,user_description,user_followers_count,user_friends_count,posts_count,user_listed_count,user_follower_handles,user_friend_handles
0,00dl3-aibfan.bsky.social,did:plc:5fwnpskl6zvknaikbb5bwb3s,Demi-Pansexual Crossiaunt Creature Thingy (DPC...,75,103,1281,NaN,"[""chocoletine.bsky.social"", ""praisethespawn.bs...","[""praisethespawn.bsky.social"", ""redisok.bsky.s..."
1,022770.bsky.social,did:plc:4pon6zyd7tyyp7gpjbktus4j,NaN,186,132,1327,NaN,"[""nanainhavana.bsky.social"", ""sannehbaky.bsky....","[""electroboyusa.bsky.social"", ""mika-brzezinski..."
2,0331online.bsky.social,did:plc:6dzaq2nkgwf5shgbchsnjwfw,USMC Vet 2/5 1st Mar Div.\n\nBanned With Trump...,35,14,708,NaN,"[""thegigibunnyz.bsky.social"", ""marth-doll.bsky...","[""darylgibson.bsky.social"", ""mschloey.bsky.soc..."
3,10bmnews.bsky.social,did:plc:hdte5leshsk5zxfnovynif3o,"10bmnews-Latest News Updates, Breaking News, ...",435,663,65398,NaN,"[""lemonsourest.bsky.social"", ""clippycoin.bsky....","[""astraea-mk.bsky.social"", ""knownforms.bsky.so..."
4,1692witchywoman.bsky.social,did:plc:jqs2i2olt22acffnaqtggcb4,WICCAN with NO filter!!!BULLDOGS rule!! Footba...,28524,41619,3742,NaN,"[""tashheadricknyc2.bsky.social"", ""scottgable.b...","[""lilgay.club"", ""tntnle.bsky.social"", ""scottga..."


In [ ]:
# Colab-ready: per-user counts of mentions & hashtags from Bluesky JSONL
import json
from collections import defaultdict
from pathlib import Path
import pandas as pd

# ---- Paths (adjust if needed) ----
JSONL_PATH = Path("/content/drive/MyDrive/Nour/bluesky/exports/filtered_posts_full.jsonl")
OUT_CSV    = Path("/content/drive/MyDrive/Nour/bluesky/exports/user_mention_hashtag_counts.csv")

# (Optional) mount Drive in Colab first:
# from google.colab import drive
# drive.mount('/content/drive')

if not JSONL_PATH.exists():
    raise FileNotFoundError(f"JSONL not found: {JSONL_PATH}")

# --- Counters ---
total_posts = defaultdict(int)
mentions_total = defaultdict(int)
hashtags_total = defaultdict(int)
posts_with_mentions = defaultdict(int)
posts_with_hashtags = defaultdict(int)

def count_features_in_post(post_record):
    """Return (mentions_in_post, hashtags_in_post) from a single post's record."""
    m = 0
    h = 0
    if not isinstance(post_record, dict):
        return m, h
    facets = post_record.get("facets") or []
    for facet in facets:
        if not isinstance(facet, dict):
            continue
        for feat in (facet.get("features") or []):
            if not isinstance(feat, dict):
                continue
            t = feat.get("py_type")
            if t == "app.bsky.richtext.facet#mention":
                m += 1
            elif t == "app.bsky.richtext.facet#tag":
                # field name is "tag" inside the feature; we just count it
                h += 1
    return m, h

# --- Scan JSONL ---
with JSONL_PATH.open("r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            item = json.loads(line)
        except json.JSONDecodeError:
            continue

        post = item.get("post", {}) or {}
        author = post.get("author", {}) or {}
        did = author.get("did")
        if not isinstance(did, str) or not did:
            continue

        total_posts[did] += 1

        record = post.get("record", {}) or {}
        m, h = count_features_in_post(record)

        # Totals
        mentions_total[did] += m
        hashtags_total[did] += h

        # Per-post presence flags
        if m > 0:
            posts_with_mentions[did] += 1
        if h > 0:
            posts_with_hashtags[did] += 1

# --- Build DataFrame ---
users = sorted(set(total_posts.keys()) |
               set(mentions_total.keys()) |
               set(hashtags_total.keys()))

rows = []
for did in users:
    tp = total_posts.get(did, 0)
    mt = mentions_total.get(did, 0)
    ht = hashtags_total.get(did, 0)
    pwm = posts_with_mentions.get(did, 0)
    pwh = posts_with_hashtags.get(did, 0)
    rows.append({
        "user_did": did,
        "total_posts": tp,
        "mentions_total": mt,
        "hashtags_total": ht,
        "posts_with_mentions": pwm,
        "posts_with_hashtags": pwh,
        "avg_mentions_per_post": round(mt / tp, 4) if tp else 0.0,
        "avg_hashtags_per_post": round(ht / tp, 4) if tp else 0.0,
    })

df = pd.DataFrame(rows).sort_values(["mentions_total", "hashtags_total", "total_posts"], ascending=[False, False, False])
df.to_csv(OUT_CSV, index=False)

print(f"Users found: {len(df)}")
print(f"Saved per-user mention/hashtag counts to: {OUT_CSV}")
print(df.head(10))


Users found: 1661
Saved per-user mention/hashtag counts to: /content/drive/MyDrive/Nour/bluesky/exports/user_mention_hashtag_counts.csv
                              user_did  total_posts  mentions_total  \
950   did:plc:mi7if3czciowfal5dqpipmdq           63              33   
858   did:plc:kqsdfw4cj532lp7bidtdzyyz           10               4   
1254  did:plc:sefl5pb7r6pbfwraona2yenj            8               3   
776   did:plc:jgr3yvurykh27i7trnjurqcv            3               2   
409   did:plc:cdhvqw6exypbvi3gfbh4cvqq            2               2   
645   did:plc:gugr5p7eshw2bfepcak7iip5            1               2   
544   did:plc:ethtgjszwmeussjrvcoequ6r            5               2   
533   did:plc:eiyryfczicuhk7xrjppyqjxv            4               2   
479   did:plc:dmqulsznwisuleirj5ea36o2            3               2   
189   did:plc:5vake3jq6c27ytc52m5rpwv5            1               2   

      hashtags_total  posts_with_mentions  posts_with_hashtags  \
950             

# **genrate the csv from the jsonl file**

In [ ]:
# Colab-ready: extract selected fields from Bluesky JSONL -> CSV
import json
import csv
from pathlib import Path

# --- Paths (edit if needed) ---
JSONL_PATH = Path("/content/drive/MyDrive/Nour/bluesky/exports/filtered_posts_full.jsonl")
OUT_CSV    = Path("/content/drive/MyDrive/Nour/bluesky/exports/filtered_posts_full.csv")

In [ ]:
# --- Helpers ---
def dig(obj, path, default=None):
    """Safely get nested value by dotted path; returns default if missing."""
    cur = obj
    for part in path.split("."):
        if cur is None:
            return default
        if isinstance(cur, dict):
            cur = cur.get(part)
        else:
            # non-dict encountered along the way
            return default
    return cur if cur is not None else default

def normalize_labels(node):
    """
    Convert post.record.labels (or similar) into a compact string.
    Accepts shapes:
      - [{"val": "x"}, ...]
      - {"values": [{"val": "x"}, ...]}
      - plain string or list of strings
    Returns a '|' separated string of label values.
    """
    vals = []
    if node is None:
        return ""
    if isinstance(node, dict):
        if "values" in node and isinstance(node["values"], list):
            for v in node["values"]:
                if isinstance(v, dict) and "val" in v:
                    vals.append(str(v["val"]))
        elif "val" in node:
            vals.append(str(node["val"]))
        elif "label" in node:
            vals.append(str(node["label"]))
    elif isinstance(node, list):
        for v in node:
            if isinstance(v, dict) and "val" in v:
                vals.append(str(v["val"]))
            elif isinstance(v, dict) and "label" in v:
                vals.append(str(v["label"]))
            elif isinstance(v, str):
                vals.append(v)
    elif isinstance(node, str):
        vals.append(node)
    return "|".join(sorted(set([x.strip() for x in vals if str(x).strip()])))

def normalize_tags(node):
    """
    Convert post.record.tags into a compact string.
    Often tags may be a simple list of strings; if dicts, try 'tag'/'val'.
    """
    vals = []
    if node is None:
        return ""
    if isinstance(node, list):
        for v in node:
            if isinstance(v, str):
                vals.append(v)
            elif isinstance(v, dict):
                if "tag" in v: vals.append(str(v["tag"]))
                elif "val" in v: vals.append(str(v["val"]))
    elif isinstance(node, dict):
        # occasionally {"values":[...]} shapes
        values = node.get("values")
        if isinstance(values, list):
            for v in values:
                if isinstance(v, str):
                    vals.append(v)
                elif isinstance(v, dict):
                    if "tag" in v: vals.append(str(v["tag"]))
                    elif "val" in v: vals.append(str(v["val"]))
    elif isinstance(node, str):
        vals.append(node)
    return "|".join(sorted(set([x.strip() for x in vals if str(x).strip()])))

def to_str(x):
    """Stringify scalars/objects consistently for CSV (without breaking)."""
    if x is None:
        return ""
    if isinstance(x, (str, int, float, bool)):
        return str(x)
    # fallback: compact JSON
    return json.dumps(x, ensure_ascii=False)

# --- Columns requested (left as-is to match your names) ---
COLUMNS = [
    "post.author.created_at",
    "post.author.did",
    "post.author.display_name",
    "post.author.handle",
    "post.author.verification",
    "post.cid",
    "post.indexed_at",
    "post.like_count",
    "post.quote_count",
    "post.record.created_at",
    "post.record.labels",
    "post.record.reply",
    "post.reply.parent.author.did",
    "post.record.reply.parent.uri",
    "post.record.reply.parent.cid",
    "post.record.tags",
    "post.record.text",
    "post.reply_count",
    "post.repost_count",
    "post.uri",
    "reason",
]

# Note: your list mixes 'post.' and 'Post.' prefixes.
# We'll try exact paths first; if a 'Post.' path is missing, we also try the lowercase 'post.' equivalent.

def get_with_case_fallback(item, path):
    val = dig(item, path, None)
    if val is not None:
        return val
    # If path starts with 'Post.', try 'post.' as a fallback
    if path.startswith("Post."):
        alt = "post." + path[len("Post."):]
        return dig(item, alt, None)
    return None

# Special cases for parent author uri/cid:
# In ATProto postView, parent has its own 'uri' and 'cid' at 'post.reply.parent.uri/cid'
# The user's requested paths say '...parent.author.uri/cid'. We'll fill with the parent-level values if author.* is missing.

def get_parent_author_uri(item):
    return dig(item, "post.record.reply.parent.uri")

def get_parent_author_cid(item):
    return dig(item, "post.record.reply.parent.cid")


# --- Read JSONL and write CSV ---
if not JSONL_PATH.exists():
    raise FileNotFoundError(f"JSONL not found: {JSONL_PATH}")

OUT_CSV.parent.mkdir(parents=True, exist_ok=True)

with JSONL_PATH.open("r", encoding="utf-8") as fin, OUT_CSV.open("w", encoding="utf-8", newline="") as fout:
    writer = csv.writer(fout)
    writer.writerow(COLUMNS)

    for line in fin:
        line = line.strip()
        if not line:
            continue
        try:
            item = json.loads(line)
        except json.JSONDecodeError:
            continue

        row = []

        for col in COLUMNS:
            # handle special normalizations
            if col == "post.record.labels":
                val = get_with_case_fallback(item, "post.record.labels")
                row.append(normalize_labels(val))
                continue

            if col == "post.record.tags":
                val = get_with_case_fallback(item, "post.record.tags")
                row.append(normalize_tags(val))
                continue

            if col == "post.reply.parent.uri":
                row.append(to_str(get_parent_author_uri(item)))
                continue

            if col == "post.reply.parent.cid":
                row.append(to_str(get_parent_author_cid(item)))
                continue

            # default: safe dig with case fallback
            val = get_with_case_fallback(item, col)
            row.append(to_str(val))

        writer.writerow(row)

print(f"Done. Wrote CSV to: {OUT_CSV}")


Done. Wrote CSV to: /content/drive/MyDrive/Nour/bluesky/exports/filtered_posts_full.csv


In [ ]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/Nour/bluesky/exports/filtered_posts_full.csv")

In [ ]:
df['post.record.reply.parent.uri'].unique()

array([nan,
       'at://did:plc:t6ubj2wlhc34awzcymh3qpur/app.bsky.feed.post/3lqbhxfn2h22k',
       'at://did:plc:4lx6nur5wstwoc4wtgj56kyu/app.bsky.feed.post/3lr7iiga2hs23',
       ...,
       'at://did:plc:z43kijz2hzz5dzow3kndcfm7/app.bsky.feed.post/3lplfbigmvk2v',
       'at://did:plc:fgw7xrzkqm2gaocjnjxyieon/app.bsky.feed.post/3lsbpmblcp22a',
       'at://did:plc:ci7cygkxnbgskitltc47ztjd/app.bsky.feed.post/3lratk2dgik2i'],
      dtype=object)

In [ ]:
import re
# Regex to extract DID from URI
URI_DID_RE = re.compile(r"^at://([^/]+)/")

def extract_did(uri):
    if isinstance(uri, str):
        m = URI_DID_RE.match(uri)
        if m:
            return m.group(1)
    return None

# Fill the column: if it's empty, derive from uri
df["post.reply.parent.author.did"] = df.apply(
    lambda row: row["post.reply.parent.author.did"]
    if pd.notna(row["post.reply.parent.author.did"]) and str(row["post.reply.parent.author.did"]).strip() != ""
    else extract_did(row["post.record.reply.parent.uri"]),
    axis=1
)

# Save back to CSV (overwrite or new file)

df.to_csv("/content/drive/MyDrive/Nour/bluesky/exports/filtered_posts_full.csv", index=False)


In [ ]:
df

,post.author.created_at,post.author.did,post.author.display_name,post.author.handle,post.author.verification,post.cid,post.indexed_at,post.like_count,post.quote_count,post.record.created_at,...,post.record.reply,post.reply.parent.author.did,post.record.reply.parent.uri,post.record.reply.parent.cid,post.record.tags,post.record.text,post.reply_count,post.repost_count,post.uri,reason
0,2025-02-12T17:55:18.147Z,did:plc:5fwnpskl6zvknaikbb5bwb3s,《☆Idiot Art☆》- offline,00dl3-aibfan.bsky.social,NaN,bafyreifqd6b4yu2hw2waujfreihycwijba4svuo5zxm5r...,2025-03-26T03:14:52.500Z,1,0,2025-03-26T03:14:51.096Z,...,NaN,None,NaN,NaN,NaN,Im shedding hair lol,1,0,at://did:plc:5fwnpskl6zvknaikbb5bwb3s/app.bsky...,NaN
1,2024-11-14T03:47:02.281Z,did:plc:4pon6zyd7tyyp7gpjbktus4j,The Grand Lady,022770.bsky.social,NaN,bafyreibydru2vwkqymtfifoy2ck3toimry6eyvysxdarf...,2025-05-29T10:00:04.249Z,0,0,2025-05-29T10:00:02.309Z,...,"{""parent"": {""cid"": ""bafyreihlotga6lblrhrebp3gm...",did:plc:t6ubj2wlhc34awzcymh3qpur,at://did:plc:t6ubj2wlhc34awzcymh3qpur/app.bsky...,bafyreihlotga6lblrhrebp3gmffma5uuq47sqvsxss5oc...,NaN,FBI investigating a virus. I don’t care where ...,0,0,at://did:plc:4pon6zyd7tyyp7gpjbktus4j/app.bsky...,NaN
2,2024-12-11T23:42:22.843Z,did:plc:6dzaq2nkgwf5shgbchsnjwfw,Anonymous Patriot,0331online.bsky.social,NaN,bafyreiavsvrmm2cgj7ia4ke7taetp2g5odpvcwcuazf7z...,2025-04-01T09:40:59.100Z,0,0,2025-04-01T09:40:55.725Z,...,NaN,None,NaN,NaN,NaN,"“The W.H.O., with Bill Gates’ support and part...",0,0,at://did:plc:6dzaq2nkgwf5shgbchsnjwfw/app.bsky...,NaN
3,2024-12-11T23:42:22.843Z,did:plc:6dzaq2nkgwf5shgbchsnjwfw,Anonymous Patriot,0331online.bsky.social,NaN,bafyreibofhrw537k42b6dzgf4w2tdmtzzxrtpajg7dzx6...,2025-03-24T01:21:18.152Z,0,0,2025-03-24T01:21:16.850Z,...,NaN,None,NaN,NaN,NaN,"""Nobody was forced to take the vaccine.""",0,0,at://did:plc:6dzaq2nkgwf5shgbchsnjwfw/app.bsky...,NaN
4,2025-03-05T18:50:42.345Z,did:plc:hdte5leshsk5zxfnovynif3o,10bmnews,10bmnews.bsky.social,NaN,bafyreihscptirmjxdc5gu2pt23lgtirqlbx2kymnw2qjc...,2025-06-27T17:53:31.941Z,0,0,2025-06-27T17:53:29Z,...,NaN,None,NaN,NaN,NaN,Millions of children at risk as global vaccina...,0,0,at://did:plc:hdte5leshsk5zxfnovynif3o/app.bsky...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3448,2024-11-22T19:42:42.755Z,did:plc:rzevoar7dyrcm57k2xivayil,NaN,ziggystardust1276.bsky.social,NaN,bafyreih7y3hn733yo524yet4hjohjjbdaa25nybduf4c3...,2025-06-23T18:29:36.336Z,1,0,2025-06-23T18:29:34.490Z,...,"{""parent"": {""cid"": ""bafyreibt35dmleotb5rrosqg7...",did:plc:fgw7xrzkqm2gaocjnjxyieon,at://did:plc:fgw7xrzkqm2gaocjnjxyieon/app.bsky...,bafyreibt35dmleotb5rrosqg7unxpbx3bfsxtiqibezap...,NaN,Are you 5 years old? No such thing as vaccine ...,2,0,at://did:plc:rzevoar7dyrcm57k2xivayil/app.bsky...,NaN
3449,2024-11-22T19:42:42.755Z,did:plc:rzevoar7dyrcm57k2xivayil,NaN,ziggystardust1276.bsky.social,NaN,bafyreigc2qsqbvac4kkd3xzhztd455sq32alzooqzc6no...,2025-06-11T02:10:04.153Z,0,0,2025-06-11T02:10:01.693Z,...,"{""parent"": {""cid"": ""bafyreidliuearmiigz2cbeba5...",did:plc:ci7cygkxnbgskitltc47ztjd,at://did:plc:ci7cygkxnbgskitltc47ztjd/app.bsky...,bafyreidliuearmiigz2cbeba5i4r2zwfabpbfgc2g2ahc...,NaN,And the testing for vaccines is Al on the up a...,0,0,at://did:plc:rzevoar7dyrcm57k2xivayil/app.bsky...,NaN
3450,2024-11-12T22:53:20.908Z,did:plc:e6nnyq6cx4yvtwzfutayed5g,Liz Furler,zilenaj.bsky.social,NaN,bafyreig5ynk6r5oyi4qyfidwvihc3iudqly4frueagas2...,2025-05-20T01:37:30.452Z,1,0,2025-05-20T01:37:27.824Z,...,NaN,None,NaN,NaN,NaN,open.substack.com/pub/maryanne... Excellent su...,0,1,at://did:plc:e6nnyq6cx4yvtwzfutayed5g/app.bsky...,NaN
3451,2024-11-17T21:15:11.527Z,did:plc:6j277n6u3p7ovagzxgwxar5h,NaN,zinemaniac.bsky.social,NaN,bafyreif3pmg4ugmolgp5hcoktbertglxilbiqe2ilhb6s...,2025-05-10T19:54:41.646Z,0,0,2025-05-10T19:54:39.622Z,...,NaN,None,NaN,NaN,NaN,#BigPharma #Plandemic #Covid19 #Vaccines #Poli...,0,0,at://did:plc:6j277n6u3p7ovagzxgwxar5h/app.bsky...,NaN


In [ ]:
df.columns

Index(['post.author.created_at', 'post.author.did', 'post.author.display_name',
       'post.author.handle', 'post.author.verification', 'post.cid',
       'post.indexed_at', 'post.like_count', 'post.quote_count',
       'post.record.created_at', 'post.record.labels', 'post.record.reply',
       'post.reply.parent.author.did', 'post.record.reply.parent.uri',
       'post.record.reply.parent.cid', 'post.record.tags', 'post.record.text',
       'post.reply_count', 'post.repost_count', 'post.uri', 'reason'],
      dtype='object')

In [ ]:
df['post.author.verification'].unique()

array([nan,
       '{"trusted_verifier_status": "none", "verifications": [{"created_at": "2025-04-21T10:47:10.429Z", "is_valid": true, "issuer": "did:plc:z72i7hdynmk6r22z27h6tvur", "uri": "at://did:plc:z72i7hdynmk6r22z27h6tvur/app.bsky.graph.verification/3lndpulv3532x", "py_type": "app.bsky.actor.defs#verificationView"}], "verified_status": "valid", "py_type": "app.bsky.actor.defs#verificationState"}',
       '{"trusted_verifier_status": "none", "verifications": [{"created_at": "2025-07-15T20:03:19.348Z", "is_valid": true, "issuer": "did:plc:z72i7hdynmk6r22z27h6tvur", "uri": "at://did:plc:z72i7hdynmk6r22z27h6tvur/app.bsky.graph.verification/3ltzp4cnu5v2e", "py_type": "app.bsky.actor.defs#verificationView"}], "verified_status": "valid", "py_type": "app.bsky.actor.defs#verificationState"}',
       '{"trusted_verifier_status": "none", "verifications": [{"created_at": "2025-06-19T10:54:31.200Z", "is_valid": true, "issuer": "did:plc:z72i7hdynmk6r22z27h6tvur", "uri": "at://did:plc:z72i7hdynm

In [ ]:
import pandas as pd
import numpy as np
import re

# --- Helpers ---
def is_empty_cell(x) -> bool:
    """Treat NaN/None/''/'null'/'None'/{} / [] as empty."""
    if pd.isna(x):
        return True
    if isinstance(x, str):
        s = x.strip().lower()
        return s in ("", "null", "none", "{}", "[]")
    return False

URI_DID_RE = re.compile(r"^at://([^/]+)/")
def extract_did_from_uri(uri: str):
    if isinstance(uri, str):
        m = URI_DID_RE.match(uri)
        if m:
            return m.group(1)
    return np.nan

# --- 1) post_type ---
reply_col = "post.record.reply" if "post.record.reply" in df.columns else None
if reply_col is None:
    # If the reply field wasn't exported, default to NaN (= treated as original)
    df["post_type"] = "original"
else:
    df["post_type"] = np.where(df[reply_col].apply(is_empty_cell), "original", "reply")

# --- 2) in_reply_to_id (prefer post.record.reply.uri, else fall back to post.reply.parent.uri) ---
if "post.record.reply.parent.uri" in df.columns:
    df["in_reply_to_id"] = df["post.record.reply.parent.uri"]
else:
    df["in_reply_to_id"] = np.nan  # no source available

# --- 3) in_reply_to_user_id (prefer post.reply.parent.author.did; else derive from URI) ---
if "post.reply.parent.author.did" in df.columns:
    df["in_reply_to_user_id"] = df["post.reply.parent.author.did"]
else:
    df["in_reply_to_user_id"] = np.nan

# If any are missing, try to derive from the reply URI we just set
mask_missing_did = df["in_reply_to_user_id"].isna() | (df["in_reply_to_user_id"].astype(str).str.strip() == "")
df.loc[mask_missing_did, "in_reply_to_user_id"] = df.loc[mask_missing_did, "in_reply_to_id"].apply(extract_did_from_uri)

# --- Save ---
df.to_csv("/content/drive/MyDrive/Nour/bluesky/exports/filtered_posts_full.csv", index=False)
print(f"Saved: {OUT_CSV}")

# Optional: quick check
print(df[["post_type", "in_reply_to_id", "in_reply_to_user_id"]].head())


✅ Saved: /content/drive/MyDrive/Nour/bluesky/exports/filtered_posts_full.csv
  post_type                                     in_reply_to_id  \
0  original                                                NaN   
1     reply  at://did:plc:t6ubj2wlhc34awzcymh3qpur/app.bsky...   
2  original                                                NaN   
3  original                                                NaN   
4  original                                                NaN   

                in_reply_to_user_id  
0                               NaN  
1  did:plc:t6ubj2wlhc34awzcymh3qpur  
2                               NaN  
3                               NaN  
4                               NaN  


In [ ]:
len(df['post.author.verification'].unique())

36

In [ ]:
import pandas as pd
import json

# Load you
def check_verified(cell):
    if pd.isna(cell) or str(cell).strip() == "":
        return "unknown"
    try:
        data = json.loads(cell) if isinstance(cell, str) else cell
    except Exception:
        return "unknown"

    # Safely check for verifications list
    verifications = data.get("verifications", []) if isinstance(data, dict) else []
    if not verifications:
        return "unknown"

    # If any verification is_valid true → verified
    for v in verifications:
        if isinstance(v, dict) and v.get("is_valid") is True:
            return "verified"
    return "not_verified"

# Create new column
df["user_verified"] = df["post.author.verification"].apply(check_verified)

# Save to new file (or overwrite if you want)
out_path = "/content/drive/MyDrive/Nour/bluesky/exports/filtered_posts_full_verified.csv"
df.to_csv(out_path, index=False)



✅ Saved with new 'user_verified' column: /content/drive/MyDrive/Nour/bluesky/exports/filtered_posts_full_verified.csv
  post.author.verification user_verified
0                      NaN       unknown
1                      NaN       unknown
2                      NaN       unknown
3                      NaN       unknown
4                      NaN       unknown


In [ ]:
df['user_verified'].unique()

array(['unknown', 'verified', 'not_verified'], dtype=object)

In [ ]:
df_post_level = df
df_user_level = pd.read_csv("/content/drive/MyDrive/Nour/bluesky/user_level/filtered_users_from_jsonl.csv")
df_features_count = pd.read_csv("/content/drive/MyDrive/Nour/bluesky/exports/user_mention_hashtag_counts.csv")

In [ ]:
df_post_level.columns

Index(['post.author.created_at', 'post.author.did', 'post.author.display_name',
       'post.author.handle', 'post.author.verification', 'post.cid',
       'post.indexed_at', 'post.like_count', 'post.quote_count',
       'post.record.created_at', 'post.record.labels', 'post.record.reply',
       'post.reply.parent.author.did', 'post.record.reply.parent.uri',
       'post.record.reply.parent.cid', 'post.record.tags', 'post.record.text',
       'post.reply_count', 'post.repost_count', 'post.uri', 'reason',
       'post_type', 'in_reply_to_id', 'in_reply_to_user_id', 'user_verified'],
      dtype='object')

In [ ]:
df_user_level.columns

Index(['user_handle', 'user_did', 'user_description', 'user_followers_count',
       'user_friends_count', 'posts_count', 'user_listed_count',
       'user_follower_handles', 'user_friend_handles'],
      dtype='object')

In [ ]:
df_features_count.columns

Index(['user_did', 'total_posts', 'mentions_total', 'hashtags_total',
       'posts_with_mentions', 'posts_with_hashtags', 'avg_mentions_per_post',
       'avg_hashtags_per_post'],
      dtype='object')

In [ ]:
import pandas as pd


merged = df_post_level.merge(
    df_user_level,
    how="left",
    left_on="post.author.did",
    right_on="user_did",
    suffixes=("", "_user")
)

merged = merged.merge(
    df_features_count,
    how="left",
    on="user_did",   # both already use user_did
    suffixes=("", "_features")
)

out_path = "/content/drive/MyDrive/Nour/bluesky/exports/merged_dataset.csv"
merged.to_csv(out_path, index=False)

print("Shape:", merged.shape)
print(merged.head())


Shape: (3964, 41)
     post.author.created_at                   post.author.did  \
0  2025-02-12T17:55:18.147Z  did:plc:5fwnpskl6zvknaikbb5bwb3s   
1  2024-11-14T03:47:02.281Z  did:plc:4pon6zyd7tyyp7gpjbktus4j   
2  2024-12-11T23:42:22.843Z  did:plc:6dzaq2nkgwf5shgbchsnjwfw   
3  2024-12-11T23:42:22.843Z  did:plc:6dzaq2nkgwf5shgbchsnjwfw   
4  2025-03-05T18:50:42.345Z  did:plc:hdte5leshsk5zxfnovynif3o   

  post.author.display_name        post.author.handle post.author.verification  \
0   《☆Idiot Art☆》- offline  00dl3-aibfan.bsky.social                      NaN   
1           The Grand Lady        022770.bsky.social                      NaN   
2        Anonymous Patriot    0331online.bsky.social                      NaN   
3        Anonymous Patriot    0331online.bsky.social                      NaN   
4                 10bmnews      10bmnews.bsky.social                      NaN   

                                            post.cid  \
0  bafyreifqd6b4yu2hw2waujfreihycwijba4svuo5zxm5

In [ ]:
merged.columns

Index(['post.author.created_at', 'post.author.did', 'post.author.display_name',
       'post.author.handle', 'post.author.verification', 'post.cid',
       'post.indexed_at', 'post.like_count', 'post.quote_count',
       'post.record.created_at', 'post.record.labels', 'post.record.reply',
       'post.reply.parent.author.did', 'post.record.reply.parent.uri',
       'post.record.reply.parent.cid', 'post.record.tags', 'post.record.text',
       'post.reply_count', 'post.repost_count', 'post.uri', 'reason',
       'post_type', 'in_reply_to_id', 'in_reply_to_user_id', 'user_verified',
       'user_handle', 'user_did', 'user_description', 'user_followers_count',
       'user_friends_count', 'posts_count', 'user_listed_count',
       'user_follower_handles', 'user_friend_handles', 'total_posts',
       'mentions_total', 'hashtags_total', 'posts_with_mentions',
       'posts_with_hashtags', 'avg_mentions_per_post',
       'avg_hashtags_per_post'],
      dtype='object')

In [ ]:
df = df.rename(columns={
    "post.author.created_at": "user_created_at",
    "post.author.did": "user_id",
    "post.author.display_name": "user_screen_name",
    "post.author.handle": "user_name",
    "post.like_count": "favorite_count",
    "post.record.created_at": "created_at",
    "post.reply.parent.author.did": "user_id",
    "post.record.tags": "hashtags",
    "post.record.text": "text",
    "post.uri": "id",
    "reason": "reason",

})